# Practical Exercise 7.02: Fama-French 3-Factor Model

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf
import zipfile
import requests
from io import BytesIO

# Define the stocks and the date range
symbols = ['AAPL', 'AMZN', 'META', 'GOOGL', 'MSFT', 'NVDA', 'TSLA']
start_date = '2014-01-03'
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')

# Download stock price data from Yahoo Finance
def download_stock_data(symbols, start_date, end_date):
    data = yf.download(symbols, start=start_date, end=end_date, auto_adjust=False, actions=False)['Adj Close']
    return data

stock_data = download_stock_data(symbols, start_date, end_date)

# Calculate daily returns
stock_returns = stock_data.pct_change().dropna() * 100

# Clean the Fama-French data
def clean_fama_french_data(file_content):
    lines = file_content.decode('utf-8').splitlines()
    start_line = next(i for i, line in enumerate(lines) if line.strip() and line[0].isdigit())
    data = "\n".join(lines[start_line:])
    df = pd.read_csv(BytesIO(data.encode('utf-8')), skip_blank_lines=True, index_col=0)
    return df

# Download Fama-French 3-factor data
url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip'
response = requests.get(url)
with zipfile.ZipFile(BytesIO(response.content)) as z:
    file_name = z.namelist()[0]
    with z.open(file_name) as f:
        ff_factors = clean_fama_french_data(f.read())

# Rename columns and process dates
ff_factors.columns = ['Mkt-RF', 'SMB', 'HML', 'RF']
ff_factors.index = pd.to_datetime(ff_factors.index, format='%Y%m%d', errors='coerce')
ff_factors = ff_factors.dropna()
ff_factors = ff_factors.loc[start_date:end_date]

# Ensure both indexes are tz-naive
stock_returns.index = stock_returns.index.tz_localize(None)
ff_factors.index = ff_factors.index.tz_localize(None)

# Get the common set of dates between returns and factors data
common_dates = stock_returns.index.intersection(ff_factors.index)

# Filter both DataFrames by the common dates
stock_returns = stock_returns.loc[common_dates]
ff_factors = ff_factors.loc[common_dates]

# Subtract the risk-free rate (RF) to get excess returns
excess_returns = stock_returns.subtract(ff_factors['RF'], axis=0)

# Function to fit the Fama-French 3-factor model
results = {}
def fama_french_3factor_regression(stock_returns, factors):
    betas = pd.DataFrame(index=stock_returns.columns, columns=['Alpha', 'Mkt-RF', 'SMB', 'HML'])
    for stock in stock_returns.columns:
        Y = stock_returns[stock]  # Stock returns
        X = factors[['Mkt-RF', 'SMB', 'HML']]  # Fama-French 3 factors
        X = sm.add_constant(X)  # Add constant term
        model = sm.OLS(Y, X).fit()
        results[stock] = model
        betas.loc[stock, 'Alpha'] = model.params['const']
        betas.loc[stock, ['Mkt-RF', 'SMB', 'HML']] = model.params[1:]  # Exclude constant
    return betas

factors = ff_factors[['Mkt-RF', 'SMB', 'HML']]
betas_table = fama_french_3factor_regression(excess_returns, ff_factors)

# Display the betas table
print("Betas Table:")
print(betas_table)

# Calculate estimated returns based on the 3-factor model, including RF
def calculate_annualized_returns(betas_table, factors):
    avg_factors = factors.mean()  # Average daily values of the factors
    trading_days = 252  # Approximate number of trading days in a year

    # Annualize the average daily factor values
    avg_factors_annualized = avg_factors * trading_days

    # Calculate annualized returns for each stock
    annualized_returns = pd.DataFrame(index=betas_table.index, columns=['Total Return'])
    for stock in betas_table.index:
        rf_annualized = avg_factors_annualized['RF']  # Annualized risk-free rate
        factor_contributions = (betas_table.loc[stock, betas_table.columns[1:]] * avg_factors_annualized[betas_table.columns[1:]]).sum()
        total_return = rf_annualized + factor_contributions

        annualized_returns.loc[stock, 'Total Return'] = total_return

    return annualized_returns



# Calculate annualized returns
annualized_returns = calculate_annualized_returns(betas_table, ff_factors)

# Display annualized returns
print("\nAnnualized Returns:")
print(annualized_returns)

# Decompose annualized returns into factor contributions
def decompose_returns(betas_table, factors):
    avg_factors = factors.mean()  # Average daily values of the factors
    trading_days = 252  # Approximate number of trading days in a year

    # Annualize the average daily factor values
    avg_factors_annualized = avg_factors * trading_days

    # Create a DataFrame to store contributions
    decomposition = pd.DataFrame(index=betas_table.index, columns=factors.columns[:-1].tolist() + ['RF', 'Total'])

    for stock in betas_table.index:
        rf_annualized = avg_factors_annualized['RF']  # Annualized risk-free rate
        factor_contributions = betas_table.loc[stock, betas_table.columns[1:]] * avg_factors_annualized[betas_table.columns[1:]]

        # Populate the DataFrame
        decomposition.loc[stock, factors.columns[:-1]] = factor_contributions
        decomposition.loc[stock, 'RF'] = rf_annualized
        decomposition.loc[stock, 'Total'] = rf_annualized + factor_contributions.sum()

    return decomposition

# Calculate the decomposition
returns_decomposition = decompose_returns(betas_table, ff_factors)

# Display the decomposition
print("\nReturns Decomposition:")
print(returns_decomposition)
